In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
from scipy.signal import find_peaks
from scipy.signal import peak_widths
import pandas as pd
import os

In [2]:
def get_image_list(directory:str, image_type:str) -> list:
    filenames = glob.glob(f"{directory}/*.{image_type}")
    
    return [os.path.normpath(f) for f in filenames]

def get_processed_image(image_path, trial, process_name, invert = False):
    filename = os.path.basename(image_path)
    image =  cv2.imread(f"./output/trial_{trial}/processed_images/masks/{process_name}/{os.path.basename(image_path)}", cv2.IMREAD_GRAYSCALE)
    
    if invert:
        image = cv2.bitwise_not(image)
        
    return image

def get_subdirectories(directory):
    return [f.path for f in os.scandir(directory) if f.is_dir()]

def get_process_names(directory):
    return [os.path.basename(f) for f in get_subdirectories(directory)]

def load_images_for_given_process(image_path, trial_number, process_name):
    """
        Load images from the given filenames. Returns raw image and the processed image.
        
        The processed image is inverted so that the lumen/stroma is black and the membrane as white.
        This is to aid in the contouring process, which sees the area of the image as the area within the contour.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    p_image = cv2.imread(f"./output/trial_{trial_number}/processed_images/masks/{process_name}/{os.path.basename(image_path)}", cv2.IMREAD_GRAYSCALE)
    

    return image, cv2.bitwise_not(p_image)




def get_original_filename(image_name, metadata_filename):
    """
        Search the metadata file for the original filename of the image that the strip was taken from, and return it.
    """
    image_metadata = pd.read_csv(metadata_filename)
    
    image_name = os.path.normpath(image_name)
    print(image_name)
    
    #output\trial_1\rois\strip_101.png
    image_name = os.path.basename(image_name)
    
    #strip_101.png
    strip_number = int(image_name.strip(".png").split("_")[1])
    
    
    # if image_metadata is none, thorw an error
    if image_metadata is None:
        raise ValueError("Image metadata is None")
    
    image_df = image_metadata[image_metadata['strip'] == strip_number].to_dict(orient='records')[0]
    return image_df["filename"]


def get_image_conversion_factors(image_name:str, conversion_df_filename: str, metadata_filename: str) -> dict:
    """ 
        Returns the dict with the nm_per_pixel and pixel_per_nm values for the given image name.
    """

    image_raw_filename = get_original_filename(image_name, metadata_filename)

    conversion_df = pd.read_csv(conversion_df_filename)
    
    conversion_df['filename'] = conversion_df['filename'].map(os.path.normpath)
    
    filename = os.path.normpath(image_raw_filename)#
    
    image_conversion_factors = conversion_df[conversion_df['filename'] == filename].to_dict(orient='records')[0]

    return {"nm_per_pixel": image_conversion_factors['nm_per_pixel'], 
            "pixel_per_nm": image_conversion_factors['pixel_per_nm'], 
            "scale": image_conversion_factors['scale'],
            "scale_pixels": image_conversion_factors['scale_pixels']}


def convert_nm_to_pixel(nm_value, nm_per_pixel):
    return nm_value / nm_per_pixel

def convert_pixel_to_nm(pixel_value, pixel_per_nm):
    return pixel_value / pixel_per_nm

def create_image_dict(image_name, metadata:dict) -> dict:
    """ retrieve the images for the given image name, trial number and process name, and return them in a dictionary """
        
    # p_image has the lumen/stroma as black and the membrane as white. We want to extract the membrane
    image, p_image = load_images_for_given_process(image_name, metadata["trial_number"], metadata["process_name"])

    # start with a black image for saving the membrane contours to
    membrane_image = np.zeros_like(p_image)    
    
    # create the contours based on the processed image
    membrane_contours = get_filtered_contours(p_image, min_area=100, max_area=np.Infinity)
    
    # draw the contours on the image
    cv2.drawContours(membrane_image, membrane_contours, -1, (255, 0, 0), -1)
    
    # invert the image so that the membrane is white and the lumen/stroma is black
    lumen_image = cv2.bitwise_not(membrane_image)

    convert_dict = get_image_conversion_factors(image_name, metadata["conversion_df_filename"], metadata["metadata_filename"])

    strip_name = os.path.basename(image_name).split(".png")[0]

    image_dict = {
        "strip_name": strip_name,
        "image_name": image_name,
        "image": image,
        "p_image": p_image,
        "lumen": lumen_image,
        "membrane": membrane_image,
        "nm_per_pixel": convert_dict["nm_per_pixel"],
        "pixel_per_nm": convert_dict["pixel_per_nm"],
        "scale": convert_dict["scale"],
        "scale_pixels": convert_dict["scale_pixels"],
    }
    
    return image_dict


def get_filtered_contours(image, min_area=0, max_area=np.Infinity, contour_method : int = cv2.RETR_EXTERNAL, contour_approximation : int = cv2.CHAIN_APPROX_SIMPLE):
    """
        Calculate the contours of the white regions of the image, then filter the results
        according to the given min and max area. Return the filtered contours.
        
    """
    
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    contours, hierarchy = cv2.findContours(image, contour_method, contour_approximation)


    return [c for c in contours if min_area < cv2.contourArea(c) < max_area]

def calculate_peak_data(image_dict:dict, metadata: dict, peak_type: str = "membrane") -> dict:
    """
        Calculate the peaks and widths of the membrane histogram, and return the data in a dict.
    """
    image_name = image_dict["image_name"]
    nm_per_pixel = image_dict['nm_per_pixel']
    pixel_per_nm = image_dict['pixel_per_nm']
    input_image = image_dict[peak_type]
    chosen_height = metadata["chosen_height"]

    peak_data = {}

    histogram = np.sum(input_image, axis=1)

    peaks, _ = find_peaks(histogram)

    avg_peak_height = np.mean(histogram[peaks])
    half_height = avg_peak_height * chosen_height

    chosen_rel_height = half_height / avg_peak_height

    peaks, _ = find_peaks(histogram, height=half_height)

    results_half = peak_widths(histogram, peaks, rel_height=chosen_height)

    widths, width_heights, left_ips, right_ips = results_half

    peak_data["peaks"] = peaks
    peak_data["histogram"] = histogram
    peak_data["avg_peak_height"] = avg_peak_height
    peak_data["half_height"] = half_height
    peak_data["chosen_rel_height"] = chosen_rel_height
    peak_data["results_half"] = results_half
    peak_data["widths"] = widths
    peak_data["width_heights"] = width_heights
    peak_data["left_ips"] = left_ips
    peak_data["right_ips"] = right_ips
    
    return peak_data

In [3]:
# define some functions to help with extracting the peaks and widths from the histograms
selected_trial = 1

metadata = {"trial_number": selected_trial,
            "process_name": "otsu_thresholding_blurred_not_equalized",
            "conversion_df_filename" : "./metadata/image_scale_conversion.csv",
            "metadata_filename" : "output/trial_1/081624_rois_metadata_bignine.csv",
            "images": get_image_list(directory=f"./output/trial_{selected_trial}/rois", image_type="png"),
            "chosen_height": 0.5,
            }


# setup the output
os.makedirs(f"./output/trial_{metadata["trial_number"]}/histograms/", exist_ok=True)

# verify we are loading the correct images
print(metadata["images"])

['output\\trial_1\\rois\\strip_101.png', 'output\\trial_1\\rois\\strip_106.png', 'output\\trial_1\\rois\\strip_134.png', 'output\\trial_1\\rois\\strip_135.png', 'output\\trial_1\\rois\\strip_161.png', 'output\\trial_1\\rois\\strip_176.png', 'output\\trial_1\\rois\\strip_187.png', 'output\\trial_1\\rois\\strip_229.png', 'output\\trial_1\\rois\\strip_232.png']


In [4]:

def calculate_grana_height(membrane_data:dict) -> float:
    """
        Calculate the height of the grana stacks in px. Take the min of the left_ips and the
        max of the right_ips.
    """
        # Left ips: [24.36666667 34.17391304 48.41463415 62.39393939 77.22580645 91.818181[82]
    # Right ips: [29.16666667 43.40625    57.33333333 71.04166667 86.8        96.5       ]
    return np.max(membrane_data["right_ips"]) - np.min(membrane_data["left_ips"])


def calculate_repeat_distance(membrane_data:dict) -> float:
    """
        Calculate the distance between the membrane peaks in nm. We need to first calculate the
        float value for the center of the peaks, then calculate the difference between the peaks.
    """
    
    # todo: instead of calculating the diff from peaks, we
    # can calculate the diff from the left_ips and right_ips of the membrane. 
    # aget the center of the peaks using the left_ips and right_ips

    left_ips = membrane_data["left_ips"]
    right_ips = membrane_data["right_ips"]
    
    # zip together as pairs
    peaks = np.array(list(zip(left_ips, right_ips)))

    # the mean of each pair will get us the center of the peak. 
    center_points = np.mean(peaks, axis=1)
    
    # calculate the differences between the peaks
    repeat_distances = np.diff(center_points)
    
    # round them down to two significant figures
    repeat_distances = np.round(repeat_distances, 2)
    
    # flatten the numpy array into a list
    repeat_distances = repeat_distances.flatten()
        
    return repeat_distances

# calculate_values(membrane_data, lumen_data, image_dict, metadata)/
def calculate_grana_values(membrane_data: dict, lumen_data: dict, image_dict: dict) -> dict:
    """ 
        Take the peaks and widths of the membrane and lumen histograms, and return the grana values.
        peaks data:
            peak_data["peaks"] = peaks
            peak_data["histogram"] = histogram
            peak_data["avg_peak_height"] = avg_peak_height
            peak_data["half_height"] = half_height
            peak_data["chosen_rel_height"] = chosen_rel_height
            peak_data["results_half"] = results_half
            peak_data["widths"] = widths
            peak_data["width_heights"] = width_heights
            peak_data["left_ips"] = left_ips
            peak_data["right_ips"] = right_ips
        image_dict:
            "image_name": image_name,
            "image": image,
            "p_image": p_image,
            "lumen": lumen_image,
            "membrane": membrane_image,
            "nm_per_pixel": convert_dict["nm_per_pixel"],
            "pixel_per_nm": convert_dict["pixel_per_nm"],
            "scale": convert_dict["scale"],
            "scale_pixels": convert_dict["scale_pixels"],
    """
    # strip,grana_height,grana_height_nm,num_lumen,repeat_distance,repeat_distance_nm,px_per_nm,nm_per_px, scale, scale_pixels
    # Also, include for each lumen and membrane: width, 
    grana_values = {
        "strip_num": os.path.basename(image_dict["image_name"]).strip(".png").strip("strip_"),
        "image_name": image_dict["image_name"],
        "image_dict": image_dict,
        "membrane_data": membrane_data,
        "lumen_data": lumen_data,
        "grana_height": calculate_grana_height(membrane_data),
        "num_lumen": len(lumen_data["peaks"]),
        "repeat_distance": calculate_repeat_distance(membrane_data),
        "lumen_width": lumen_data["widths"],
        "membrane_width": membrane_data["widths"],
    }
    
    return grana_values

def plot_histogram(grana_data: dict, metadata:dict, peak_type: str = "membrane", display: bool = False):

    image_dict = grana_data["image_dict"]
    membrane_data = grana_data["membrane_data"]
    lumen_data = grana_data["lumen_data"]
    strip_name = image_dict["strip_name"]
    process_name = metadata["process_name"]
    trial_number = metadata["trial_number"]
    
    if peak_type == "membrane":
        peaks = membrane_data["peaks"]
        histogram = membrane_data["histogram"]
        half_height = membrane_data["half_height"]
        left_ips = membrane_data["left_ips"]
        right_ips = membrane_data["right_ips"]
    else:
        peaks = lumen_data["peaks"]
        histogram = lumen_data["histogram"]
        half_height = lumen_data["half_height"]
        left_ips = lumen_data["left_ips"]
        right_ips = lumen_data["right_ips"]

    ################ Plot it ################
    # plot the histogram, but make sure the background is white
    fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150, facecolor='w')
    plt.plot(histogram)

    # plot the peaks, and then a green line dropping down
    plt.plot(peaks, histogram[peaks], "x")
    for peak in peaks:
        plt.plot([peak, peak], [0, histogram[peak]], "--g")
        
    # use the left and right interpolated points to plot the width of the peak
    for left_ip, right_ip in zip(left_ips, right_ips):
        plt.plot([left_ip, right_ip], [half_height, half_height], "-r")

    plt.title(f"{peak_type} Histogram\n{image_name}")
    plt.savefig(f"./output/trial_{trial_number}/histograms/{strip_name}_{peak_type}_histogram.png")
    
    if display:
        plt.show()
    else:
        plt.close()

In [5]:
# strip,grana_height,grana_height_nm,num_lumen,repeat_distance,repeat_distance_nm,px_per_nm,nm_per_px, scale, scale_pixels
# Include the scale and scale_pixels in the data export. 
# Also, include for each lumen and membrane: width, 
import pandas as pd
import numpy as np

def export_grana_values(grana_data):
    return {
        "strip": grana_values['strip_num'],
        "grana_height" : grana_values['grana_height'],
        "grana_height_nm": grana_values['grana_height'] * grana_values['image_dict']['nm_per_pixel'],
        "num_lumen": grana_values['num_lumen'],
        "repeat_distance": grana_values['repeat_distance'],
        "repeat_distance_nm": np.round(grana_values['repeat_distance'] * grana_values['image_dict']['nm_per_pixel'], 2),
        "px_per_nm": grana_values['image_dict']['pixel_per_nm'],
        "nm_per_px": grana_values['image_dict']['nm_per_pixel'],
        "scale": grana_values['image_dict']['scale'],
        "scale_pixels": grana_values['image_dict']['scale_pixels'],
        "lumen_width": grana_values['lumen_width'],
        "membrane_width": grana_values['membrane_width'],
    }


all_grana = []
all_grana_data= []

for image_number, image_name in enumerate(metadata["images"]):

    # load the images in their dict: image, p_image, lumen_image, membrane_image    
    image_dict = create_image_dict(image_name, metadata)

    membrane_data = calculate_peak_data(image_dict, metadata, peak_type="membrane")
    lumen_data = calculate_peak_data(image_dict, metadata, peak_type="lumen")
    
    grana_values = calculate_grana_values(membrane_data, lumen_data, image_dict)
    
    all_grana_data.append(export_grana_values(grana_values))
    all_grana.append(grana_values)    
    

# create a dataframe from the grana data
grana_df = pd.DataFrame(all_grana_data)


output\trial_1\rois\strip_101.png
output\trial_1\rois\strip_106.png
output\trial_1\rois\strip_134.png
output\trial_1\rois\strip_135.png
output\trial_1\rois\strip_161.png
output\trial_1\rois\strip_176.png
output\trial_1\rois\strip_187.png
output\trial_1\rois\strip_229.png
output\trial_1\rois\strip_232.png


In [6]:
# Create a dataframe from the grana data
grana_df = pd.DataFrame(all_grana_data)

# drop any columns that are not needed
grana_df = grana_df.drop(columns=["repeat_distance_nm", "grana_height_nm"])

# add a column for identifying the membrane or lumen type
grana_df["type"] = "-"


# just the membranes 

In [7]:

# Split the dataframes into two separate dataframes
membrane_df = grana_df.drop(columns=["lumen_width", "repeat_distance"])

# set the type for each dataframe
membrane_df["type"] = "membrane"

# Add an 'index' column to track the order of each item in the 'membrane_width' list
membrane_df["index"] = membrane_df.apply(lambda row: list(range(len(row["membrane_width"]))), axis=1)

# Explode the membrane_df based on 'membrane_width' and the new 'membrane_index' column
membrane_df = membrane_df.explode(["membrane_width", "index"])

# Print the resulting dataframe
print("Membrane DataFrame with Index:")
print(membrane_df)

#save it to csv
membrane_df.to_csv(f"./output/trial_{metadata['trial_number']}/grana_data_membrane.csv", index=False)


Membrane DataFrame with Index:
   strip  grana_height  num_lumen  px_per_nm  nm_per_px  scale  scale_pixels  \
0    101     72.133333          5       0.88   1.136364    500           440   
0    101     72.133333          5       0.88   1.136364    500           440   
0    101     72.133333          5       0.88   1.136364    500           440   
0    101     72.133333          5       0.88   1.136364    500           440   
0    101     72.133333          5       0.88   1.136364    500           440   
..   ...           ...        ...        ...        ...    ...           ...   
8    232    165.230769         11       0.88   1.136364    500           440   
8    232    165.230769         11       0.88   1.136364    500           440   
8    232    165.230769         11       0.88   1.136364    500           440   
8    232    165.230769         11       0.88   1.136364    500           440   
8    232    165.230769         11       0.88   1.136364    500           440   

   membr

# just the lumens and repeat_distance

In [8]:
# Drop the 'membrane_width' column as not needed here
lumen_df = grana_df.copy()
lumen_df["type"] = "lumen"

# drop membrane_width
lumen_df = lumen_df.drop(columns=["membrane_width"])

# Add an 'index' column to track the order of each item in the 'lumen_width' list before exploding
lumen_df["index"] = lumen_df.apply(lambda row: list(range(len(row["lumen_width"]))), axis=1)

# Zip the lumen_width and repeat_distance columns together as a list of tuples, including the index
lumen_df["lumen_repeat_pairs"] = lumen_df.apply(lambda row: list(zip(row["lumen_width"], row["repeat_distance"], row["index"])), axis=1)

# Explode the 'lumen_repeat_pairs' column into separate rows
lumen_df = lumen_df.explode("lumen_repeat_pairs")

# After exploding, split the tuple back into three columns (lumen_width, repeat_distance, and index)
lumen_df["lumen_width"], lumen_df["repeat_distance"], lumen_df["index"] = zip(*lumen_df["lumen_repeat_pairs"])

# Drop the temporary 'lumen_repeat_pairs' column
lumen_df = lumen_df.drop(columns=["lumen_repeat_pairs"])

# Print the resulting dataframe
print("\nLumen DataFrame:")
print(lumen_df.head())

# Save to CSV (assuming you have 'metadata' dict with 'trial_number')
metadata = {'trial_number': 1}  # example metadata
lumen_df.to_csv(f"./output/trial_{metadata['trial_number']}/grana_data_lumen.csv", index=False)


Lumen DataFrame:
  strip  grana_height  num_lumen  repeat_distance  px_per_nm  nm_per_px  \
0   101     72.133333          5            12.02       0.88   1.136364   
0   101     72.133333          5            14.08       0.88   1.136364   
0   101     72.133333          5            13.84       0.88   1.136364   
0   101     72.133333          5            15.30       0.88   1.136364   
0   101     72.133333          5            12.15       0.88   1.136364   

   scale  scale_pixels  lumen_width   type  index  
0    500           440     5.007246  lumen      0  
0    500           440     5.008384  lumen      1  
0    500           440     5.060606  lumen      2  
0    500           440     6.184140  lumen      3  
0    500           440     5.018182  lumen      4  


# to combine them, we have to create the missing columns for each


In [9]:
# Ensure both dataframes have the same columns by adding missing columns with NaN
lumen_df["membrane_width"] = pd.NA  # membrane_width doesn't exist in lumen_df
membrane_df["lumen_width"] = pd.NA  # lumen_width doesn't exist in membrane_df
membrane_df["repeat_distance"] = pd.NA  # repeat_distance doesn't exist in membrane_df

# Concatenate the dataframes
combined_df = pd.concat([lumen_df, membrane_df], ignore_index=True, sort=False)

# Optionally, you can sort by 'strip' and reset the index if needed
combined_df = combined_df.sort_values(by=['strip', 'index']).reset_index(drop=True)

# Print the combined dataframe
print("\nCombined DataFrame:")
print(combined_df)

# Optionally, save the combined dataframe to a CSV file
combined_df.to_csv(f"./output/trial_{metadata['trial_number']}/grana_data_combined.csv", index=False)


Combined DataFrame:
    strip  grana_height  num_lumen  repeat_distance  px_per_nm  nm_per_px  \
0     101     72.133333          5            12.02       0.88   1.136364   
1     101     72.133333          5              NaN       0.88   1.136364   
2     101     72.133333          5            14.08       0.88   1.136364   
3     101     72.133333          5              NaN       0.88   1.136364   
4     101     72.133333          5            13.84       0.88   1.136364   
..    ...           ...        ...              ...        ...        ...   
118   232    165.230769         11            15.37       0.88   1.136364   
119   232    165.230769         11              NaN       0.88   1.136364   
120   232    165.230769         11            12.09       0.88   1.136364   
121   232    165.230769         11              NaN       0.88   1.136364   
122   232    165.230769         11              NaN       0.88   1.136364   

     scale  scale_pixels  lumen_width      type index 

C:\Users\woodm\AppData\Local\Temp\ipykernel_2536\3654444863.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([lumen_df, membrane_df], ignore_index=True, sort=False)


In [10]:
import pandas as pd
import numpy as np

def explode_grana_data(df) -> (pd.DataFrame, pd.DataFrame):
    """
    Explode the grana data! We need one row per lumen_width and repeat_distance in one df (dropping the membrane_width),
    and one row per membrane_width in another df (dropping the lumen_width and repeat_distance).
    """
    
    # Split the dataframes into two separate dataframes
    membrane_df = df.drop(columns=["lumen_width", "repeat_distance", "repeat_distance_nm"])
    lumen_df = df.drop(columns=["membrane_width", "repeat_distance_nm"])
    
    # Explode the membrane_df based on 'membrane_width'
    membrane_df = membrane_df.explode("membrane_width")
    
    # Zip the lumen_width and repeat_distance columns together as a list of tuples
    lumen_df["lumen_repeat_pairs"] = list(zip(lumen_df["lumen_width"], lumen_df["repeat_distance"]))
    
    # Explode the 'lumen_repeat_pairs' column into separate rows
    lumen_df = lumen_df.explode("lumen_repeat_pairs")
    
    # After exploding, split the tuple back into two columns
    lumen_df["lumen_width"], lumen_df["repeat_distance"] = zip(*lumen_df["lumen_repeat_pairs"])
    
    # Drop the temporary 'lumen_repeat_pairs' column
    lumen_df = lumen_df.drop(columns=["lumen_repeat_pairs"])
    
    return membrane_df, lumen_df

# create a dataframe from the grana data
grana_df = pd.DataFrame(all_grana_data)

# Example: Call the function with your grana DataFrame
membrane_df, lumen_df = explode_grana_data(grana_df)

# save the dataframes to csv
membrane_df.to_csv(f"./output/trial_{metadata['trial_number']}/membrane_data.csv", index=False)
lumen_df.to_csv(f"./output/trial_{metadata['trial_number']}/lumen_data.csv", index=False)

print(lumen_df.head())

ValueError: too many values to unpack (expected 2)

# We have all the grana values, but we don't have a good way to export them in a way that makes sense in csv format.
We need to convert the grana values into a format that can be easily exported to a csv file.


```
Peak_num 0: peak: 5, peak_height: 15810, width: 5.27, left base: 2.18, right base: 7.45, center: 4.82
Peak_num 1: peak: 18, peak_height: 15810, width: 5.16, left base: 15.26, right base: 20.43, center: 17.84
Peak_num 2: peak: 31, peak_height: 15810, width: 4.82, left base: 28.89, right base: 33.70, center: 31.30
Peak_num 3: peak: 44, peak_height: 15810, width: 5.56, left base: 42.00, right base: 47.56, center: 44.78
Peak_num 4: peak: 59, peak_height: 15810, width: 5.00, left base: 56.69, right base: 61.69, center: 59.19
Peak_num 5: peak: 72, peak_height: 15810, width: 4.95, left base: 69.72, right base: 74.67, center: 72.20
Peak_num 6: peak: 86, peak_height: 15810, width: 5.09, left base: 83.22, right base: 88.32, center: 85.77
Peak_num 7: peak: 100, peak_height: 15810, width: 4.81, left base: 97.53, right base: 102.34, center: 99.94
```

# Plot the membrane histogram and its parameters

In [ ]:
display = False

for i, grana_data in enumerate(all_grana):
    image_dict = grana_data["image_dict"]
    membrane_data = grana_data["membrane_data"]
    lumen_data = grana_data["lumen_data"]
    image_name = image_dict["image_name"]
    strip_name = image_dict["strip_name"]
    process_name = metadata["process_name"]
    
    print(f"Processing {image_dict['image_name']}")
    
    plot_histogram(grana_data, metadata, peak_type="membrane", display=display)
    plot_histogram(grana_data, metadata, peak_type="lumen", display=display)
    
    # calculate the grana height
    grana_height = calculate_grana_height(membrane_data)
    
    left_ips = membrane_data["left_ips"]
    right_ips = membrane_data["right_ips"]

    if display:
        print(f"{strip_name} grana height: {grana_height} px")
        print(f"Left ips: {left_ips}")
        print(f"Right ips: {right_ips}")
    

strip,grana_height,grana_height_nm,num_lumen,repeat_distance,repeat_distance_nm,px_per_nm,nm_per_px, scale, scale_pixels
Include the scale and scale_pixels in the data export. 
Also, include for each lumen and membrane: width, 